This chapter focuses on tools to help combine, join, and rearrange data

# Hierarchical indexing

Hierarchical indexing is an important feature of pandas that enables you to have multiple (two or more) index levels on an axis. 
Somewhat abstractly, it provides a way for you to work with higher dimensional data in a lower dimensional form. 

In [6]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.889423
   2    1.245490
   3    0.356754
b  1    2.002424
   3   -0.843689
c  1   -0.016966
   2   -0.339703
d  2    1.118601
   3    0.536543
dtype: float64

_The “gaps” in the index display mean “use the label directly above”:_

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

With a hierarchically indexed object, so-called partial indexing is possible, enabling
you to concisely select subsets of the data:

In [4]:
data['b']

1    2.002424
3   -0.843689
dtype: float64

In [5]:
data['b':'c']

b  1    2.002424
   3   -0.843689
c  1   -0.016966
   2   -0.339703
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1    2.002424
   3   -0.843689
d  2    1.118601
   3    0.536543
dtype: float64

Selection is even possible from an “inner” level:

In [7]:
data.loc[:, 2]

a    1.245490
c   -0.339703
d    1.118601
dtype: float64

Hierarchical indexing plays an important role in reshaping data and group-based
operations like forming a pivot table. For example, you could rearrange the data into
a DataFrame using its unstack method:

In [8]:
 data.unstack()

,1,2,3
a,-0.889423,1.245490,0.356754
b,2.002424,NaN,-0.843689
c,-0.016966,-0.339703,NaN
d,NaN,1.118601,0.536543


The inverse operation of unstack is stack:

In [9]:
data.unstack().stack()

a  1   -0.889423
   2    1.245490
   3    0.356754
b  1    2.002424
   3   -0.843689
c  1   -0.016966
   2   -0.339703
d  2    1.118601
   3    0.536543
dtype: float64

With a DataFrame, either axis can have a hierarchical index:

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

The hierarchical levels can have names (as strings or any Python objects). If so, these
will show up in the console output:

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

With partial column indexing you can similarly select groups of columns:


In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## Reordering and Sorting Levels


At times you will need to rearrange the order of the levels on an axis or sort the data
by the values in one specific level. The _swaplevel_ takes two level numbers or names
and returns a new object with the levels interchanged (but the data is otherwise
unaltered):

In [13]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

_sort_index_, on the other hand, sorts the data using only the values in a single level.
When swapping levels, it’s not uncommon to also use _sort_index_ so that the result is
lexicographically sorted by the indicated level:

In [14]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [15]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

__Data selection performance is much better on hierarchically
indexed objects if the index is lexicographically sorted starting with
the outermost level—that is, the result of calling
sort_index(level=0) or sort_index().__

## Summary Statistics by Level


Many descriptive and summary statistics on DataFrame and Series have a level
option in which you can specify the level you want to aggregate by on a particular
axis. 

In [16]:
# frame.sum(level='key2')
# frame.sum(level='color', axis=1)

## Indexing with a DataFrame’s columns


It’s not unusual to want to use one or more columns from a DataFrame as the row
index; alternatively, you may wish to move the row index into the DataFrame’s columns. 

In [17]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame’s _set_index_ function will create a new DataFrame using one or more of
its columns as the index:

In [18]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

By default the columns are removed from the DataFrame, though you can leave them
in:

In [19]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

_reset_index_, on the other hand, does the opposite of _set_index_; the hierarchical
index levels are moved into the columns:

In [20]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# Combining and Merging Datasets

Data contained in pandas objects can be combined together in a number of ways:
- _pandas.merge_ connects rows in DataFrames based on one or more keys. This
will be familiar to users of SQL or other relational databases, as it implements
database join operations.
- _pandas.concat_ concatenates or “stacks” together objects along an axis.
- The _combine_first_ instance method enables splicing together overlapping data
to fill in missing values in one object with values from another.

## Database-Style DataFrame Joins

Merge or join operations combine datasets by linking rows using one or more keys.
These operations are central to relational databases (e.g., SQL-based). The merge
function in pandas is the main entry point for using these algorithms on your data.

In [21]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [22]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

,key,data2
0,a,0
1,b,1
2,d,2


This is an example of a many-to-one join; the data in df1 has multiple rows labeled a
and b, whereas df2 has only one row for each value in the key column. Calling merge
with these objects we obtain:

In [23]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


If the column to join on is not specified, merge uses the overlapping column names as the keys. 
It’s a good practice to specify explicitly, though:

In [24]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


If the column names are different in each object, you can specify them separately:

In [25]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})

pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


By default merge does an 'inner' join; the keys in the result are the intersection, or the common set found in both tables. 
Other possible options are 'left','right', and 'outer'. 
The outer join takes the union of the keys, combining the effect of applying both left and right joins:

In [26]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,a,2.0,0.0
1,a,4.0,0.0
2,a,5.0,0.0
3,b,0.0,1.0
4,b,1.0,1.0
5,b,6.0,1.0
6,c,3.0,NaN
7,d,NaN,2.0


#### Different join types with how argument

- _'inner'_ Use only the key combinations observed in both tables
- _'left'_ Use all key combinations found in the left table
- _'right'_ Use all key combinations found in the right table
- _'output'_ Use all key combinations observed in both tables together

Many-to-many merges have well-defined, though not necessarily intuitive, behavior.

In [27]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [28]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [29]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Many-to-many joins form the Cartesian product of the rows. Since there were three
'b' rows in the left DataFrame and two in the right one, there are six 'b' rows in the
result. The join method only affects the distinct key values appearing in the result:

In [30]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


To merge with multiple keys, pass a list of column names:


In [31]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,bar,one,3.0,6.0
1,bar,two,NaN,7.0
2,foo,one,1.0,4.0
3,foo,one,1.0,5.0
4,foo,two,2.0,NaN


To determine which key combinations will appear in the result depending on the
choice of merge method, think of the multiple keys as forming an array of tuples to
be used as a single join key (even though it’s not actually implemented that way).


__When you’re joining columns-on-columns, the indexes on the
passed DataFrame objects are discarded.__

_merge_ has a __suffixes__ option for specifying strings to append
to overlapping names in the left and right DataFrame objects:

In [32]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [33]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### merge function arguments

- _left_ DataFrame to be merged on the left side.
  - _right_ DataFrame to be merged on the right side.
- _on_ Column names to join on. Must be found in both DataFrame objects. If not specified and no other join keys
given, will use the intersection of the column names in left and right as the join keys
a in each row
- _left_on_ Columns in left DataFrame to use as join keysy' in r
- _copy_ If False, avoid copying data into resulting data structure in some exceptional cases; by default always
copies.
- _indicator_ Adds a special column _merge that indicates the source of each row; values will be 'left_only',
'right_only', or 'both' based on the origin of the joined data in each row.esult).

## Merging on Index

In some cases, the merge key(s) in a DataFrame will be found in its index. In this
case, you can pass left_index=True or right_index=True (or both) to indicate that
the index should be used as the merge key:

In [34]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [35]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [36]:
right1

,group_val
a,3.5
b,7.0


In [37]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


Since the default merge method is to intersect the join keys, you can instead form the
union of them with an outer join:


In [38]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


With hierarchically indexed data, things are more complicated, as joining on index is
implicitly a multiple-key merge:

In [39]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])

In [40]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [41]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In this case, you have to indicate multiple columns to merge on as a list (note the
handling of duplicate index values with how='outer')

In [42]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [43]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
         right_index=True, how='outer')

,key1,key2,data,event1,event2
4,Nevada,2000,NaN,2.0,3.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0


Using the indexes of both sides of the merge is also possible:


In [44]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [45]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [46]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [47]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a convenient join instance for merging by index. It can also be used
to combine together many DataFrame objects having the same or similar indexes but
non-overlapping columns. In the prior example, we could have written:

In [48]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


For simple index-on-index merges, you can pass a list of DataFrames to join as
an alternative to using the more general concat function

In [49]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                        index=['a', 'c', 'e', 'f'],
                        columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [50]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [51]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


## Concatenating Along an Axis

In the context of pandas objects such as Series and DataFrame, having labeled axes
enable you to further generalize array concatenation. In particular, you have a number of additional things to think about:
- If the objects are indexed differently on the other axes, should we combine the distinct elements in these axes or use only the shared values (the intersection)?
- Do the concatenated chunks of data need to be identifiable in the resulting object?
- Does the “concatenation axis” contain data that needs to be preserved? In many cases, the default integer labels in a DataFrame are best discarded during
concatenation.


The _concat_ function in pandas provides a consistent way to address each of these
concerns. 

In [52]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default concat works along axis=0, producing another Series. If you pass axis=1,
the result will instead be a DataFrame (axis=1 is the columns):

In [53]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In this case there is no overlap on the other axis, which as you can see is the sorted
union (the 'outer' join) of the indexes. You can instead intersect them by passing
join='inner':

In [54]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [55]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [56]:
# 'f' and 'g' labels disappear because of the join='inner' option
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


A potential issue is that the concatenated pieces are not identifiable in the result. 
Suppose instead you wanted to create a hierarchical index on the concatenation axis. Todo this, use the keys argument:

In [57]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [58]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


In the case of combining Series along axis=1, the keys become the DataFrame column headers:

In [59]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


The same logic extends to DataFrame objects:


In [60]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])

In [61]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [62]:
df2

,three,four
a,5,6
c,7,8


In [63]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

If you pass a dict of objects instead of a list, the dict’s keys will be used for the keys
option:


In [64]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

We can name the created axis levels with the names argument:

In [65]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

 DataFrames in which the row index does not contain
any relevant data:

In [66]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [67]:
df1

,a,b,c,d
0,-0.473766,0.002685,-1.743090,1.542872
1,1.514530,-0.722351,2.043923,-1.543427
2,0.226615,-0.334619,0.956847,0.566795


In [68]:
df2

,b,d,a
0,0.222134,-1.662642,1.000511
1,0.515493,-1.099548,-0.084971


In this case, you can pass _ignore_index=True_:

In [69]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,-0.473766,0.002685,-1.743090,1.542872
1,1.514530,-0.722351,2.043923,-1.543427
2,0.226615,-0.334619,0.956847,0.566795
3,1.000511,0.222134,NaN,-1.662642
4,-0.084971,0.515493,NaN,-1.099548


### concat function arguments

## Combining Data with Overlap


You may have two datasets whose indexes overlap in full
or part.

In [70]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b.iloc[-1] = np.nan

In [71]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [72]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [73]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

Series has a _combine_first_ method, which performs the equivalent of this operation
along with pandas’s usual data alignment logic:


In [74]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

With DataFrames, _combine_first_ does the same thing column by column, so you
can think of it as “patching” missing data in the calling object with data from the
object you pas:

In [75]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [76]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [77]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [78]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


#  Reshaping and Pivoting

## Reshaping with Hierarchical Indexing


Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:
- ___stack___
This “rotates” or pivots from the columns in the data to the rows
- ___unstack___
This pivots from the rows into the columns

In [79]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                                     name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Using the stack method on this data pivots the columns into the rows, producing a
Series:

In [80]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

From a hierarchically indexed Series, you can rearrange the data back into a Data‐
Frame with unstack:


In [81]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the innermost level is unstacked (same with stack). You can unstack a different level by passing a level number or name:

In [82]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [83]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren’t found 
in each of the subgroups:

In [84]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [85]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


Stacking filters out missing data by default, so the operation is more easily invertible:

In [86]:
 data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [87]:
data2.unstack().stack(future_stack=True)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

When you unstack in a DataFrame, the level unstacked becomes the lowest level in
the result

In [88]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [89]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

When calling stack, we can indicate the name of the axis to stack:


In [90]:
df.unstack('state').stack('side')

C:\Users\Eliud\AppData\Local\Temp\ipykernel_11236\3997798123.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack('state').stack('side')


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

# <span style="color: var(--vscode-foreground);"><b>Aggregation</b>&nbsp;</span>

In [1]:
import pandas as pd

data = {
    'Category': ['A', 'A', 'B', 'B', 'C', 'C'],
    'Values': [10, 20, 30, 40, 50, 60]
}

df = pd.DataFrame(data)
df

,Category,Values
0,A,10
1,A,20
2,B,30
3,B,40
4,C,50
5,C,60


## <span style="color: var(--vscode-foreground);">Basic Aggregation</span>

In [2]:
print("Sum:\n", df['Values'].sum())
print("Mean:\n", df['Values'].mean())
print("Median:\n", df['Values'].median())
print("Standard Deviation:\n", df['Values'].std())
print("Variance:\n", df['Values'].var())
print("Minimum:\n", df['Values'].min())
print("Maximum:\n", df['Values'].max())

Sum:
 210
Mean:
 35.0
Median:
 35.0
Standard Deviation:
 18.708286933869708
Variance:
 350.0
Minimum:
 10
Maximum:
 60


## <span style="color: var(--vscode-foreground);">GroupBy Operations</span>

<span style="color: var(--vscode-foreground);">Grouping data and then performing aggregation on the groups is a powerful technique for summarizing data.</span>

In [3]:
grouped = df.groupby('Category')

print("Group Sum:\n", grouped.sum())
print("Group Mean:\n", grouped.mean())
print("Group Count:\n", grouped.count())

Group Sum:
           Values
Category        
A             30
B             70
C            110
Group Mean:
           Values
Category        
A           15.0
B           35.0
C           55.0
Group Count:
           Values
Category        
A              2
B              2
C              2


## <span style="color: var(--vscode-foreground);">Multiple Aggregations</span>

<span style="color: var(--vscode-foreground);">You can apply multiple aggregation functions at once using the <code>agg</code> method.</span>

In [4]:
print("Multiple Aggregations:\n", grouped['Values'].agg(['sum', 'mean', 'count']))

Multiple Aggregations:
           sum  mean  count
Category                  
A          30  15.0      2
B          70  35.0      2
C         110  55.0      2


## Pivot Tables

<span style="color: var(--vscode-foreground);">Pivot tables are used to reshape and summarize data, allowing multi-dimensional aggregation</span>

In [7]:
pivot = df.pivot_table(values='Values', index='Category', aggfunc=np.mean)
print("Pivot Table:\n", pivot)

Pivot Table:
           Values
Category        
A           15.0
B           35.0
C           55.0


<ipython-input-7-309970ace23b>:1: FutureWarning: The provided callable <function mean at 0x000001FE1EEFC540> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'mean' instead.
  pivot = df.pivot_table(values='Values', index='Category', aggfunc=np.mean)


## <span style="color: var(--vscode-foreground);">Custom Aggregation Functions</span>

<span style="color: var(--vscode-foreground);">You can create custom aggregation functions and apply them to your data.</span>

In [8]:
def range_func(x):
    return x.max() - x.min()

print("Custom Aggregation (Range):\n", grouped['Values'].agg(range_func))

Custom Aggregation (Range):
 Category
A    10
B    10
C    10
Name: Values, dtype: int64


## <span style="color: var(--vscode-foreground);">Time Series Aggregation</span>

In [9]:
date_rng = pd.date_range(start='2023-01-01', end='2023-01-10', freq='D')
df_time = pd.DataFrame(date_rng, columns=['Date'])
df_time['Data'] = np.random.randint(0, 100, size=(len(date_rng)))

print("Time Series Data:\n", df_time)

Time Series Data:
         Date  Data
0 2023-01-01    84
1 2023-01-02    42
2 2023-01-03    15
3 2023-01-04    14
4 2023-01-05    85
5 2023-01-06    17
6 2023-01-07    63
7 2023-01-08    55
8 2023-01-09    12
9 2023-01-10    26


In [10]:
# Set the date column as index
df_time.set_index('Date', inplace=True)

# Resample and aggregate
print("Resampled Data (Sum by Week):\n", df_time.resample('W').sum())

Resampled Data (Sum by Week):
             Data
Date            
2023-01-01    84
2023-01-08   291
2023-01-15    38


## <span style="color: var(--vscode-foreground);">Sales Data Aggregation Project<br></span>

In [14]:
sales_data = {
    'Date': pd.date_range(start='2023-01-01', periods=12, freq='M'),
    'Product': ['A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C'],
    'Sales': [100, 150, 200, 130, 170, 210, 90, 140, 180, 160, 190, 220]
}

df_sales = pd.DataFrame(sales_data)
print("Sales Data:\n", df_sales)
df_sales.info()

Sales Data:
          Date Product  Sales
0  2023-01-31       A    100
1  2023-02-28       B    150
2  2023-03-31       C    200
3  2023-04-30       A    130
4  2023-05-31       B    170
5  2023-06-30       C    210
6  2023-07-31       A     90
7  2023-08-31       B    140
8  2023-09-30       C    180
9  2023-10-31       A    160
10 2023-11-30       B    190
11 2023-12-31       C    220
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     12 non-null     datetime64[ns]
 1   Product  12 non-null     object        
 2   Sales    12 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 420.0+ bytes


In [15]:
# Time Series Aggregation
df_sales.set_index('Date', inplace=True)
print("Resampled Sales Data (Quarterly):\n", df_sales.resample('Q').sum())

Resampled Sales Data (Quarterly):
            Product  Sales
Date                     
2023-03-31     ABC    450
2023-06-30     ABC    510
2023-09-30     ABC    410
2023-12-31     ABC    570


In [16]:
# Group by Product and aggregate
grouped_sales = df_sales.groupby('Product').agg({'Sales': ['sum', 'mean', 'count']})
print("Grouped Sales Data:\n", grouped_sales)

Grouped Sales Data:
         Sales             
          sum   mean count
Product                   
A         480  120.0     4
B         650  162.5     4
C         810  202.5     4


In [17]:
# Pivot Table for Monthly Sales
pivot_sales = df_sales.pivot_table(values='Sales', index='Date', columns='Product', aggfunc=np.sum)
print("Pivot Table (Monthly Sales):\n", pivot_sales)

Pivot Table (Monthly Sales):
 Product         A      B      C
Date                           
2023-01-31  100.0    NaN    NaN
2023-02-28    NaN  150.0    NaN
2023-03-31    NaN    NaN  200.0
2023-04-30  130.0    NaN    NaN
2023-05-31    NaN  170.0    NaN
2023-06-30    NaN    NaN  210.0
2023-07-31   90.0    NaN    NaN
2023-08-31    NaN  140.0    NaN
2023-09-30    NaN    NaN  180.0
2023-10-31  160.0    NaN    NaN
2023-11-30    NaN  190.0    NaN
2023-12-31    NaN    NaN  220.0


<ipython-input-17-cbd75043c612>:2: FutureWarning: The provided callable <function sum at 0x000001FE1EEEAE80> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass 'sum' instead.
  pivot_sales = df_sales.pivot_table(values='Sales', index='Date', columns='Product', aggfunc=np.sum)
